In [ ]:
# importing necessary libraries and modules.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [3]:
# defining the basic building blocks of the transformer model, the multi-head attention mechanism
# the function computes the attention between each pair of positions in a sequence.

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        # init of model input parameters
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        # init of linear trasformation layers
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
    # calculates the attention 
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
    # reshape the input tensor into multiple heads  
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
    # combines the attention output from all heads
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
    # computes the multi head self attention 
    # this allows the model to focus on some different aspect of the input sequence
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [7]:
# define position-wise feed-forward networks

In [9]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        # init of two linear transformation layers and ReLU activation function
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
    # forward applies these transformations and activation function sequentially to compute the output
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    # this process enables the model to consider the position of input elements while making predictions

In [11]:
# define positional encoding. who is used to inject the position information of each token in the input sequence.
# it uses sine and cosine functions of different frequencies to generate the positional encoding.

In [13]:
class PositionalEncoding(nn.Module):
    # init imput parameter d_model and max_seq_length who creates a tensor to store the positional encoding values
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        # calculate the sine and cosine values for even and off indices, respectively, based on the scaling factor div_term
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
    # computes the positional encoding by adding the stored positional encoding values to the input tensor
    # this allows the model to capture the position information of the input sequence
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [17]:
# build encoder layers who consists of a multi head attention layer, a position wise feed fotward layer, and two normalization layers

In [21]:
class EncoderLayer(nn.Module):
    # init input parameters and components, multi-head attention module, position wise feed-forward module, two layers normalization modules and drop out layer
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    # computes the encoder layer output by applying self-attention, adding the attention output to the input tensor, and normalizing the result
    # it computes the position wize feed-forward output combining it with normalized attention output and normalizes the final result before returning the processed tensor
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [23]:
# building decoder layer who consists of two multi head attention layers, a position wise feed forward layer and three normalization layers

In [25]:
class DecoderLayer(nn.Module):
    # init of input parameters and components, multi head attention modules, position wise feed forward module and three normalization modules and finaly a drop out layer
    # multi head attention modules are for masked self attention and cross attention.
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    # computes the decoder layer output by performing the following steps:
    # 1. calculate the masked self attention output and add it to the input tensor following by a dropout layer noralization.
    # 2. compute the cross attention output between the decoder and encoder outputs, add it to the normalized masked self attention output following by a dropout layer normalization.
    # 3. calculate the position wise feed forward output and combine it with the normalized cross attention output following by a dropout layer normalization.
    # 4. return the processed tensor.
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x
    # there operations enable the decoder to generate target sequences based on the input and encoder output.

In [27]:
# combining the necoder and decoder layers to create and complete the transformer model.

In [29]:
class Transformer(nn.Module):
    # init input parameters and embedding layers for source and target sequences, position encoding module, encoder layer and decoder layer modules, linear layer and dropout layer
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
    # funciton to create binary masks for source and target sequences to ignore padding tokens and prevent the decoder from attending to future tokens
    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask
    # computes the transformer models's output through the following steps:
    # 1. generation of source and target masks using the generate_mask method
    # 2. compute source and target embeddings and apply positional encoding and dropout
    # 3. process the source sequence through encoder layers and update the enc_output tensor
    # 4. process the target sequence through decoder layers using enc_output and masks and updating the dec_output tensor
    # 5. apply the linear projection layer to the decoder output obtaining output logits
    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output
    # with those steps we enable the transformer model to process input sequences and generate output sequences based on the combined functionality of its components

In [31]:
# preparing sample data of toy dataset for demonstration purposes.
# in real-life purposes, we would use a larger dataset, preprocess the text, and create vocabulary mappings for source and target languages

In [33]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [35]:
# training the model using the sample data
# in real-life purposes, we would use the larger dataset and split it into training and validation sets. 

In [37]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.680098533630371
Epoch: 2, Loss: 8.551651000976562
Epoch: 3, Loss: 8.484859466552734
Epoch: 4, Loss: 8.428597450256348
Epoch: 5, Loss: 8.3724946975708
Epoch: 6, Loss: 8.306314468383789
Epoch: 7, Loss: 8.228670120239258
Epoch: 8, Loss: 8.141640663146973
Epoch: 9, Loss: 8.065211296081543
Epoch: 10, Loss: 7.994351387023926
Epoch: 11, Loss: 7.909265995025635
Epoch: 12, Loss: 7.830058574676514
Epoch: 13, Loss: 7.745765209197998
Epoch: 14, Loss: 7.66180419921875
Epoch: 15, Loss: 7.579207420349121
Epoch: 16, Loss: 7.490774631500244
Epoch: 17, Loss: 7.41463041305542
Epoch: 18, Loss: 7.327287197113037
Epoch: 19, Loss: 7.248085021972656
Epoch: 20, Loss: 7.165849685668945
Epoch: 21, Loss: 7.086756229400635
Epoch: 22, Loss: 7.013087272644043
Epoch: 23, Loss: 6.930078506469727
Epoch: 24, Loss: 6.848872184753418
Epoch: 25, Loss: 6.780012607574463
Epoch: 26, Loss: 6.701689720153809
Epoch: 27, Loss: 6.626711368560791
Epoch: 28, Loss: 6.556502342224121
Epoch: 29, Loss: 6.48274850845336